In [1]:
#====================================================================================================
# kakaobrain/kogpt 모델을 이용하여 PEFT(Parameter-Efficient Fine-tuning)->LoRA(Low-Rank Adaptation) 기법으로 Q&A 파인튜닝 하는 예시
#
# 참조 DOC: https://huggingface.co/docs/peft/index
# 참조소스: https://github.com/huggingface/peft
#
# package 설치 
# peft: pip install peft
# load_in_8bit : pip install -i https://test.pypi.org/simple/ bitsandbytes-cudaXXX  (XXX는 CUDA version (e.g. 11.6 = 116))
# transfomers 4.27.1 이상으로 업데이트 : pip install -U transformers[pytorch]
# dispatch_model() got an unexpected keyword argument 'offload_index' 오류 => accelerate 업데이트 : pip install -U accelerate
# module 'bitsandbytes.nn' has no attribute 'Linear8bitLt' => pip install bitsandbytes==0.37.2
# 'MatmulLtState' object has no attribute 'memory_efficient_backward' 오류 => bitsandbytes 버전 0.37.2 설치 : pip install bitsandbytes==0.37.2
#====================================================================================================

import torch
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler 
import numpy as np
import pandas as pd
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AutoModelForCausalLM, AutoTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup

from tqdm.notebook import tqdm
import os
import time
from myutils import GPU_info, seed_everything, mlogging, SaveBERTModel, AccuracyForMLM

# 입력
model_path='../data11/model/LLM/bigscience/bloomz-7b1-mt/'
#model_path='bigscience/bloom-3b'

device = GPU_info()
print(device)

#seed 설정
seed_everything(111)

#logging 설정
logger =  mlogging(loggername="bloomz-7b1-mt-ft", logfilename="../log/bloomz-7b1-mt-ft")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../log/bloomz-7b1-mt-ft_2023-06-09.log


In [2]:
# tokenizer 로딩 
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [3]:
# 모델 정의 하고, embedding size를 tokenizer 사이즈만큼 조정
from transformers import AutoModelForCausalLM
from torch import float32, nn, exp


# 참고 소스 : https://github.com/jeremyarancio/llm-rpg/blob/main/llm/training_utils.py
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): 
        return super().forward(x).to(float32)

    
def prepare_model(model):
    for param in model.parameters():
        param.requires_grad = False  # freeze the model - train adapters later
        if param.ndim == 1:
            # cast the small parameters (e.g. layernorm) to fp32 for stability
            param.data = param.data.to(float32)
    model.gradient_checkpointing_enable()  # reduce number of stored activations
    model.enable_input_require_grads()
    model.lm_head = CastOutputToFloat(model.lm_head)
    return model

'''
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    return f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
'''
'''
#model = GPT2LMHeadModel.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
  model_path, revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device=device, non_blocking=True)

model.resize_token_embeddings(len(tokenizer))

model.to(device)
'''

# 모델 로딩
model = AutoModelForCausalLM.from_pretrained(model_path, return_dict=True, load_in_8bit=True, device_map="auto")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /MOCOMSYS/anaconda3/envs/bong/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 4096)
    (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear8bitLt(in_features=4096, out_features=12288, bias=True)
          (dense): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear8bitLt(in_features=4096, out_features=16384, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear8bitLt(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, eleme

In [5]:
# lora 와 연동
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

model = prepare_model(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

#peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1)

model = get_peft_model(model, lora_config)

In [6]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 4096)
        (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-29): 30 x BloomBlock(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear8bitLt(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=12288, bias=False)
                )
                (lora_embedding_A): Para

In [7]:
model.print_trainable_parameters()
#print_trainable_parameters(model)

trainable params: 7864320 || all params: 7076880384 || trainable%: 0.11112693126452029


In [8]:
# 데이터 셋을 생성한다.
# => 문장뒤에 </s> 추가.
from datasets import Dataset
#corpus_path = "../data11/korpora/kowiki_20190620/wiki_20190620_small.txt"
corpus_path = "../data11/ai_hub/vs1/IT.과학.txt"
texts = ""
with open(corpus_path, 'r') as f:
    for line in f:
        if len(line) > 1:
            texts += line
        
    # 문장들에서 '\n'(띄어쓰기) -> '.</s>'로 치환함.
    texts = texts.replace("\n", "." + tokenizer.eos_token)
 
# 데이터 셋 생성
dataset = Dataset.from_dict({'text': [texts]})

In [9]:
from typing import Callable, Mapping

# tokenizer 처리 함수
def tokenize(element: Mapping, tokenizer: Callable, context_length: int) -> str:
    inputs = tokenizer(element['text'], truncation=True, return_overflowing_tokens=True, 
                       return_length=True, max_length=context_length)
    inputs_batch = []
    for length, input_ids in zip(inputs['length'], inputs['input_ids']):
        if length == context_length: # We drop the last input_ids that are shorter than max_length
            inputs_batch.append(input_ids)
    return {"input_ids": inputs_batch}

# tokenizer dataset 생성
context_length = 2048  # 문장 길이(모델의 허용할수 있는 최대 길이 설정)
test_size = 0.001
tokenized_dataset = dataset.map(tokenize, batched=True, fn_kwargs={'tokenizer': tokenizer, 'context_length': context_length},
                                         remove_columns=dataset.column_names)

tokenized_dataset_dict = tokenized_dataset.train_test_split(test_size=test_size, shuffle=True)

print(f'len(tokenized_dataset):{len(tokenized_dataset)}')
print(f'len(tokenized_dataset_dict):{len(tokenized_dataset_dict)}')


  0%|          | 0/1 [00:00<?, ?ba/s]

len(tokenized_dataset):10415
len(tokenized_dataset_dict):2


In [10]:
# 훈련 param 설정 
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
output_dir = './output'

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=3,
    gradient_accumulation_steps=1,
    warmup_steps=100,
    weight_decay=0.1,
    learning_rate=3e-4, 
    fp16=True,
    logging_steps=1000,    # 훈련시, 로깅할 step 수 (크면 10000번 정도하고, 작으면 100번정도)
    output_dir=output_dir
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset_dict['train'],
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [ ]:
# 훈련 시작
model.config.use_cache = False  # silence warnings
trainer.train()
model.config.use_cache = True

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kobongsoo (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss


In [ ]:
# 모델 저장
### 전체모델 저장
TMP_OUT_PATH = '../data11/model/LLM/bigscience/bloomz-7b1-mt-LoRA/'
os.makedirs(TMP_OUT_PATH, exist_ok=True)
#torch.save(model, OUTPATH + 'pytorch_model.bin') 
# save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨
model.save_pretrained(TMP_OUT_PATH)

# tokeinizer 파일 저장(vocab)
VOCAB_PATH = TMP_OUT_PATH
tokenizer.save_pretrained(VOCAB_PATH)
print(f'==> save_model : {TMP_OUT_PATH}')

In [ ]:
# 여기서부터는 저장된 모델 평가하는 코드임.

In [ ]:
# 여기서부터는 저장된 모델 평가하는 코드임.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftConfig, PeftModel

# Import the model
hf_repo = '../data11/model/LLM/kakao/kogpt-float16-ft-last/'

config = PeftConfig.from_pretrained(hf_repo)
config.base_model_name_or_path = '../data11/model/LLM/kakao/kogpt-float16/'

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Lora model 불러오기
model = PeftModel.from_pretrained(model, hf_repo)

In [ ]:
model

In [ ]:

'''
수학은 기원전 600년 경에 살았던 탈레스로부터 시작됐다.
하지만 탈레스가 태어나기 전에도 수학을 연구한 사람이 있을 수도 있기 때문에, 인류의 역사와 더불어 시작되었다고 할 수 있다.
교역•분배•과세 등의 인류의 사회 생활에 필요한 모든 계산을 수학이 담당해 왔고, 농경 생활에 필수적인 천문 관측과 달력의 제정, 토지의 측량 또한 수학이 직접적으로 관여한 분야이다.
고대 수학을 크게 발전시킨 나라로는 이집트, 인도, 그리스, 중국 등이 있다.
그 중에서도 그리스는 처음으로 수학의 방정식에서 변수를 문자로 쓴 나라이다.

위 내용을 바탕으로 아래 질문에 대해 답변해 주세요
질문 : 고대 수학을 가장 크게 발전시킨 곳은?
'''
prompt ="오늘 날씨는 비가 오고 있다. 내일 날씨는 "

## Generate
max_new_tokens = 200
temperature = 0.5
do_sample = False

# Generate text
inputs = tokenizer(prompt, return_tensors="pt")

tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            #eos_token_id=tokenizer.eos_token_id,
            #early_stopping=True
        )

print(tokenizer.decode(tokens[0]))

In [ ]:
# 원본 모델 평가 해봄

In [ ]:
# 원본 모델 로딩
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftConfig, PeftModel

# Import the model
model_path = '../data11/model/LLM/kakao/kogpt-float16/'

model1 = AutoModelForCausalLM.from_pretrained(model_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer1 = AutoTokenizer.from_pretrained(model_path)

In [ ]:
model1

In [ ]:
prompt1 ="오늘 날씨는 비가 오고 있다. 내일 날씨는 "

## Generate
max_new_tokens = 200
temperature = 0.5
do_sample = False

# Generate text
inputs = tokenizer1(prompt1, return_tensors="pt")

tokens1 = model1.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            #eos_token_id=tokenizer.eos_token_id,
            #early_stopping=True
        )

print(tokenizer1.decode(tokens1[0]))